# Obteniendo el TOTAL de casos positivos, fallecidos y vacunados contra COVID-19 por cada departamento del Perú

En este notebook se busca obtener un dataset con el TOTAL con el número de **casos positivos**, **fallecidos** y **totalmente vacunados** por cada **departamento** del Perú. De la misma manera se agrega el **número de habitantes** de cada departamento, para calcular la **tasa de mortalidad por 100 000 habitantes** y el **porcentaje de vacunación** respectivo.

Se considera como **completamente vacunado** a todas las personas que recibieron 2 dosis de vacunación.

## A. Lectura y Procesamiento

En esta sección se lee y procesa el dataset **Data Product 1** ([DP1_covid19-peru_x_semanaEpi.csv](Data/DP1_covid19-peru_x_semanaEpi.csv)) para obtener un resumen general por departamento.

Datos de población estimada para el 2020 obtenida del reporte [«PERÚ:Estimaciones y Proyecciones de Población por Departamento, Provincia y Distrito, 2018 - 2020»](https://www.inei.gob.pe/media/MenuRecursivo/publicaciones_digitales/Est/Lib1715/Libro.pdf). Página 39 estimaciones de población total proyectada para 2020. Fuente: [INEI](https://www.inei.gob.pe/).

In [50]:
import pandas as pd
import dask.dataframe as dd

In [51]:
# Cargamos un diccionario con el número de habitantes por departamento en 2020
dic_dep = {
    "AMAZONAS"	:   426806,
    "ANCASH"	:   1180638,
    "APURIMAC"	:   430736,
    "AREQUIPA"	:	1497438,
    "AYACUCHO"	:   668213,
    "CAJAMARCA"	:	1453711,
    "CALLAO"	:	1129854,
    "CUSCO"	    :	1357075,
    "HUANCAVELICA":	365317,
    "HUANUCO"	:	760267,
    "ICA"	    :	975182,
    "JUNIN"	    :	1361467,
    "LA LIBERTAD":	2016771,
    "LAMBAYEQUE":	1310785,
    "LIMA"	    :	10628470,
    "LORETO"	:	1027559,
    "MADRE DE DIOS":173811,
    "MOQUEGUA"	:	192740,
    "PASCO"	    :	271904,
    "PIURA"	    :	2047954,
    "PUNO"	    :	1237997,
    "SAN MARTIN":	899648,
    "TACNA"	    :	370974,
    "TUMBES"	:	251521,
    "UCAYALI"	:	589110,
}

# Convertimos a dataframe
df_dep = pd.DataFrame(dic_dep.items(), columns = ['departamento', 'no_habitantes'])

In [52]:
# Leer el DP1
url_dp1 = 'Data/DP1_covid19-peru_x_semanaEpi.csv'
df = pd.read_csv(url_dp1)

# Procesar el df (sumar cada col)
df = pd.DataFrame(df.sum(axis = 0, skipna = True)).reset_index().rename({'index': 'var', 0: 'total'}, axis=1)

# Dejar solamente filas que contengan el nombre de un departamento
lst_dfs = [] 

for dep in df_dep['departamento']:
    temp_df = df.loc[df['var'].str.contains(dep)]
    if not temp_df.empty:
        lst_dfs.append(temp_df)
        
df = pd.concat(lst_dfs, ignore_index=True).drop_duplicates()  # Drop duplicates

# Eliminamos a vacunados (_vac) porque necesitamos todas las dosis 1, 2 y 3 no solo vacunación completa
df = df[~df['var'].str.contains('_vac')]  
    
# Separate in sub dfs and remove substrings
df_cas = df[df['var'].str.contains('_cas')].rename({'var': 'departamento','total': 'casos'}, axis = 1)
df_cas['departamento'] = df_cas['departamento'].str.replace('_cas', '')
df_fal = df[df['var'].str.contains('_fal')].rename({'var': 'departamento','total': 'fallecidos'}, axis = 1)
df_fal['departamento'] = df_fal['departamento'].str.replace('_fal', '')

# Unir en un solo dataframe el resumen por departamentos 
df_summary = df_dep.merge(df_cas, how='left', on='departamento')
df_summary = df_summary.merge(df_fal, how='left', on='departamento')

df_summary

,departamento,no_habitantes,casos,fallecidos
0,AMAZONAS,426806,33083,1289
1,ANCASH,1180638,81307,6773
2,APURIMAC,430736,26866,1528
3,AREQUIPA,1497438,118587,9770
4,AYACUCHO,668213,35182,2164
5,CAJAMARCA,1453711,67800,4194
6,CALLAO,1129854,104895,10123
7,CUSCO,1357075,76669,4836
8,HUANCAVELICA,365317,16667,1185
9,HUANUCO,760267,36993,2725


## B. Agregar el número de dosis

In [53]:
# Leer solamente "id_eess" y "dosis" del dataset de vacunados
url_vac = 'RawData/TB_VACUNACION_COVID19.csv'
vac_col = ['id_eess', 'dosis']
df_vac = dd.read_csv(url_vac, sep = ",", usecols = vac_col, dtype = {'id_eess':'int32','dosis': 'category'})

# Obtener las variables dummies de la variable "dosis"
df_vac = df_vac.categorize()
dosis = dd.get_dummies(df_vac['dosis']).drop(['4', '5'], axis = 1)

# Ahora se tiene la ubicación y el número de dosis aplicada
df_vac = dd.concat([df_vac, dosis], axis=1, ignore_unknown_divisions=True)
del dosis, df_vac['dosis']

In [54]:
# Leer los datasets de ubigeo y centros de vacunación
url_ubigeo = 'RawData/TB_UBIGEOS.csv'
url_vaccenter = 'RawData/TB_EESS.csv'
ubigeo = pd.read_csv(url_ubigeo, usecols = ['id_ubigeo', 'departamento'])
vaccenter = pd.read_csv(url_vaccenter, usecols= ['id_eess','id_ubigeo'])

# Unimos ambos dataset mediante "id_ubigeo"
vaccenter = vaccenter.merge(ubigeo, on = 'id_ubigeo', how = 'left')
del ubigeo, vaccenter['id_ubigeo']

In [55]:
# Encontrar el departamento correspondiente de todas las personas vacunadas
df_vac = df_vac.merge(vaccenter, on = 'id_eess', how = 'left')

# Contar el total de número de dosis por departamento
df_vac = df_vac.groupby('departamento').agg({'1': ['sum'], '2': ['sum'], '3': ['sum']}).compute()
df_vac = df_vac.astype({('1', 'sum'): 'int64', ('2', 'sum'): 'int64', ('3', 'sum'): 'int64'})

In [56]:
# Unir al df resumen (df_summary)
df_summary = df_summary.merge(df_vac, how='left', on='departamento')

# Rename cols
df_summary.rename(columns = {'index':'departamentos', ('1', 'sum'):'dosis_1', 
                             ('2', 'sum'): 'dosis_2', ('3', 'sum'):'dosis_3'}, inplace = True)

df_summary

,departamento,no_habitantes,casos,fallecidos,dosis_1,dosis_2,dosis_3
0,AMAZONAS,426806,33083,1289,262837,223698,36559
1,ANCASH,1180638,81307,6773,918972,851867,177289
2,APURIMAC,430736,26866,1528,343343,296213,43277
3,AREQUIPA,1497438,118587,9770,1164336,1054705,201578
4,AYACUCHO,668213,35182,2164,429579,338066,34070
5,CAJAMARCA,1453711,67800,4194,1034938,879126,96149
6,CALLAO,1129854,104895,10123,1047675,969070,226774
7,CUSCO,1357075,76669,4836,984789,848645,110218
8,HUANCAVELICA,365317,16667,1185,245100,191277,21004
9,HUANUCO,760267,36993,2725,521499,423733,65443


## C. Añadir detalles

In [57]:
# Mortality rate per 100k per department
df_summary['tasa_mortalidad'] = round((df_summary['fallecidos']/df_summary['no_habitantes'])*100000,2)

# % of people fully vaccinated per department
df_summary['vac_porcentaje'] = round((df_summary['dosis_2']*100)/df_summary['no_habitantes'],2)

df_summary

,departamento,no_habitantes,casos,fallecidos,dosis_1,dosis_2,dosis_3,tasa_mortalidad,vac_porcentaje
0,AMAZONAS,426806,33083,1289,262837,223698,36559,302.01,52.41
1,ANCASH,1180638,81307,6773,918972,851867,177289,573.67,72.15
2,APURIMAC,430736,26866,1528,343343,296213,43277,354.74,68.77
3,AREQUIPA,1497438,118587,9770,1164336,1054705,201578,652.45,70.43
4,AYACUCHO,668213,35182,2164,429579,338066,34070,323.85,50.59
5,CAJAMARCA,1453711,67800,4194,1034938,879126,96149,288.50,60.47
6,CALLAO,1129854,104895,10123,1047675,969070,226774,895.96,85.77
7,CUSCO,1357075,76669,4836,984789,848645,110218,356.35,62.53
8,HUANCAVELICA,365317,16667,1185,245100,191277,21004,324.38,52.36
9,HUANUCO,760267,36993,2725,521499,423733,65443,358.43,55.73


In [58]:
# Resumen general
print(f'Total habitantes: {df_summary.no_habitantes.sum()}')
print(f'Total fallecidos: {df_summary.fallecidos.sum()}')
print(f'Total vacunados con 1 dosis: {df_summary.dosis_1.sum()}')
print(f'Total vacunados con 2 dosis: {df_summary.dosis_2.sum()}')
print(f'Total vacunados con 3 dosis: {df_summary.dosis_3.sum()}')
print(f'Porcentaje de la población vacunada: {round(df_summary.dosis_2.sum()*100/df_summary.no_habitantes.sum(),2)}')

Total habitantes: 32625948
Total fallecidos: 202867
Total vacunados con 1 dosis: 24511772
Total vacunados con 2 dosis: 21611587
Total vacunados con 3 dosis: 4223869
Porcentaje de la población vacunada: 66.24


## D. Guardar el dataset

In [59]:
df_summary.to_csv('Data/DP2_covid19-peru_resumen_x_departamentos.csv',index = False)